In [ ]:
import numpy as np
import pandas as pd
from glob import glob 
from tqdm import tqdm
import seaborn as sns 

# for converting the text file containing the quarry locations into csv file
import csv

# for computing the geographical distance between two points 
import math


from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, auc, classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score
from datetime import datetime
import h5py
from sklearn.preprocessing import LabelEncoder
from scipy import stats, signal
from sklearn.preprocessing import StandardScaler
import obspy
from obspy.geodetics.base import gps2dist_azimuth, gps2dist_azimuth
from obspy.clients.fdsn import Client
import time
pd.set_option('display.max_columns', None)
from joblib import dump, load
from obspy.signal.filter import envelope
import tsfel


import sys
sys.path.append('../feature_extraction_scripts/physical_feature_extraction_scripts')
import seis_feature
#from seis_feature import compute_physical_features
from tsfel import time_series_features_extractor, get_features_by_domain
from datetime import timedelta
import os
import sys
sys.path.append('../common_scripts')

from common_processing_functions import apply_cosine_taper
from common_processing_functions import butterworth_filter

import pickle
from zenodo_get import zenodo_get

## Some custom helpful functions

In [ ]:
## Some helpful functions in plotting 
def interquartile(df):

    # Set the lower and upper quantile thresholds (25% and 75%)
    lower_quantile = 0.01
    upper_quantile = 0.99

    # Filter the DataFrame based on the quantile range for all columns
    filtered_df = df[
        (df >= df.quantile(lower_quantile)) &
        (df <= df.quantile(upper_quantile))
    ]

    # Drop rows with any NaN values (if needed)
    #filtered_df = filtered_df.dropna(axis = 1)

    return filtered_df


trace_cm_phy_tsf_man = []
trace_report_phy_tsf_man = []
def plot_confusion_matrix(cf = trace_cm_phy_tsf_man):

    
    annot_kws = {"fontsize": 15}
    labels = ['Precision', 'Recall', 'F1-Score']
    class_labels = ['Earthquake', 'Explosion']
    plt.figure(figsize = [8,6])

    # Set annotation font size within each block
    annot_kws = {"fontsize": 15}
    ax = sns.heatmap(cf, annot=True, cmap='Blues', fmt='d', xticklabels = class_labels, yticklabels = class_labels, annot_kws=annot_kws)


    # Set tick label font size
    ax.set_xticklabels(class_labels, fontsize=15)
    ax.set_yticklabels(class_labels, fontsize=15)


    plt.xlabel('Predicted', fontsize = 15)
    plt.ylabel('Actual', fontsize = 15)
    plt.title('Total samples: '+str(len(y_pred)), fontsize = 20)
    plt.tight_layout()



annot_kws = {"fontsize": 15}
def plot_classification_report(cr = trace_report_phy_tsf_man): 

    labels = ['Precision', 'Recall', 'F1-Score']
    class_labels = ['Earthquake', 'Explosion']
    # Set a pleasing style
    sns.set_style("whitegrid")

    
    # Create a figure and axes for the heatmap
    plt.figure(figsize = [8,6])
    ax = sns.heatmap(pd.DataFrame(cr).iloc[:3, :2], annot=True, cmap='Blues', yticklabels = labels, xticklabels=class_labels, vmin=0.8, vmax=1, annot_kws=annot_kws)

    # Set labels and title
    # Set tick label font size
    ax.set_xticklabels(class_labels, fontsize=15)
    ax.set_yticklabels(labels, fontsize=15)

    ax.set_xlabel('Metrics', fontsize=15)
    ax.set_ylabel('Classes', fontsize=15)
    ax.set_title('Classification Report', fontsize=18)

    # Create a colorbar
    #cbar = ax.collections[0].colorbar
    #cbar.set_ticks([0.5, 1])  # Set custom tick locations
    #cbar.set_ticklabels(['0', '0.5', '1'])  # Set custom tick labels

    # Adjust layout
    plt.tight_layout()

    # Show the plot
    plt.show()
    
    
    
from math import radians, sin, cos, sqrt, atan2

def calculate_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    
    # Radius of the Earth in kilometers
    R = 6371.0
    
    # Calculate the difference in latitude and longitude
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Calculate the distance using the Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    # Distance in kilometers
    distance = R * c
    
    return distance
    





## Downloading features from a repository on Zenodo

### Donwloading tsfel features from a 150s window

In [ ]:
## Downloading tsfel features


## Downloading the physical features 
## This is doi for physical features (for both 50s and 150s) - 10.5281/zenodo.10689585
## This is doi for tsfel features of 50s window length - 10.5281/zenodo.10689673
## This is doi for tsfel features of 150s window length - 10.5281/zenodo.10689824

doi = '10.5281/zenodo.10689824'  #Downloading tsfel features for 150s length
# Download files from the Zenodo record
files = zenodo_get([doi])

### Downloading physical features

In [ ]:
## Downloading the updated physical features 

doi = '10.5281/zenodo.11193585'
#Downloading physical
# Download files from the Zenodo record
files = zenodo_get([doi])

## Loading tsfel features

In [ ]:
# These waveforms are filtered between 1-10 Hz
# extracting features of surface events, thunder and sonic booms
features_exotic_tsfel = pd.read_csv('tsfel_features_exotic_50_100.csv')




# features of noise
features_noise_tsfel = pd.read_csv('tsfel_features_noise_50_100.csv')





# features of explosion
features_explosion_tsfel = pd.read_csv('tsfel_features_comcat_z_explosion_50_100.csv')



# features of earthquake (had to extract it in three parts because of memory constraints)
features_eq1 = pd.read_csv('tsfel_features_comcat_z_earthquake_50_100_part1.csv')
features_eq2 = pd.read_csv('tsfel_features_comcat_z_earthquake_50_100_part2.csv')
features_eq3 = pd.read_csv('tsfel_features_comcat_z_earthquake_50_100_part3.csv')



# features of earthquakes
features_earthquake_tsfel = pd.concat([features_eq1, features_eq2, features_eq3])

## Loading physical features

In [ ]:
# extracting features of surface events, thunder and sonic booms
features_exotic_physical = pd.read_csv('new_physical_features_exotic_z_50_100.csv')

features_surface_physical = features_exotic_physical[features_exotic_physical['source'] == 'surface']
features_sonic_physical = features_exotic_physical[features_exotic_physical['source'] == 'sonic']
features_thunder_physical = features_exotic_physical[features_exotic_physical['source'] == 'thunder']



# features of noise
features_noise_physical = pd.read_csv('new_physical_features_noise_z_50_100.csv')


# features of explosion
features_explosion_physical = pd.read_csv('new_physical_features_comcat_z_explosion_50_100.csv')

# features of earthquakes
features_eq1 = pd.read_csv('new_physical_features_earthquake_z_part1_50_100.csv')
features_eq2 = pd.read_csv('new_physical_features_earthquake_z_part2_50_100.csv')
features_eq3 = pd.read_csv('new_physical_features_earthquake_z_part3_50_100.csv')
features_eq4 = pd.read_csv('new_physical_features_earthquake_z_part4_50_100.csv')

features_earthquake_physical = pd.concat([features_eq1, features_eq2, features_eq3, features_eq4])

## Removing the large files from the directory

The features files we downloaded are very large in size. This will create problems when uploading to github repository so once we store the features in variable we will remove these files

In [ ]:
# Get the current working directory
directory = os.getcwd()

# List the files in the directory
files = os.listdir(directory)

# Iterate over the files and delete CSV files
for file in files:
    if file.endswith(".csv"):
        file_path = os.path.join(directory, file)
        os.remove(file_path)

## Merging tsfel and physical features

In [ ]:
# Each event is identified by a serial number. The serial number is the iteration number when we iterated through 
# the catalog, since tsfel and physical features were extracted separately, we have to merge them together. 
features_noise = pd.merge(features_noise_physical, features_noise_tsfel, on = 'serial_no')
features_earthquake = pd.merge(features_earthquake_physical, features_earthquake_tsfel, on = 'serial_no')
features_explosion = pd.merge(features_explosion_physical, features_explosion_tsfel, on = 'serial_no')



# The exotic events further contains three classes - (i) surface, (ii) sonic and (iii) thunder
# since sonic and thunder contain very few events, we are going to exclude them fron our analysis from now on. 
features_surface_tsfel = features_exotic_tsfel[features_exotic_tsfel['source'] == 'surface']
features_sonic_tsfel = features_exotic_tsfel[features_exotic_tsfel['source'] == 'sonic']
features_thunder_tsfel = features_exotic_tsfel[features_exotic_tsfel['source'] == 'thunder']

features_surface_physical = features_exotic_physical[features_exotic_physical['source'] == 'surface']
features_sonic_physical = features_exotic_physical[features_exotic_physical['source'] == 'sonic']
features_thunder_physical = features_exotic_physical[features_exotic_physical['source'] == 'thunder']


features_surface = pd.merge(features_surface_physical, features_surface_tsfel, on = 'serial_no')
features_sonic = pd.merge(features_sonic_physical, features_sonic_tsfel, on = 'serial_no')
features_thunder = pd.merge(features_thunder_physical, features_thunder_tsfel, on = 'serial_no')


## Concatenating all the features from all classes. 
features_all = pd.concat([features_surface, features_sonic, features_thunder, features_noise, features_explosion, features_earthquake])

### Removing correlated features

In [ ]:
serial_nos = features_all['serial_no'].values
features_all = features_all.drop(['Unnamed: 0_x','Unnamed: 0_y', 'source_x', 'serial_no'], axis = 1, errors = 'ignore')
features_all.rename(columns={'source_y': 'source'}, inplace=True)
corr_features = tsfel.correlated_features(features_all.iloc[:, 1:453])

In [ ]:
features_all.drop(corr_features, axis=1, inplace=True)
features_all['serial_no'] = serial_nos
print(f'So we have {features_all.shape[0]} events and each event have {features_all.shape[1]} features')

## Preprocessing the input features

In [ ]:
# dropping the columns that contain NaNs
features_all = features_all.dropna(axis = 1)

# dropping the rows that contains NaNs
features_all = features_all.dropna()


## dropping all the rows containing infinity values
features_all = features_all.replace([np.inf, -np.inf], np.nan).dropna()


## dropping sonic boom and thunder events
features_all = features_all[features_all['source'] != 'sonic']
features_all = features_all[features_all['source'] != 'thunder']

print(f'So we have {features_all.shape[0]} events and each event have {features_all.shape[1]} features')


## Dropping the columns that contain same values for all the events, as these features do not really contribute in classification

In [ ]:
# Check unique values in each column
unique_counts = features_all.nunique()

# Identify columns with only one unique value (same value for all rows)
single_value_columns = unique_counts[unique_counts == 1].index


# Drop columns with the same value for all rows
features_all = features_all.drop(columns=single_value_columns)

print(f'So we have {features_all.shape[0]} events and each event have {features_all.shape[1]} features')


## Removing the outliers

In [ ]:
df = features_all.drop(['serial_no', 'source'], axis = 1)
# Calculate Z-scores for each feature
z_scores = np.abs(stats.zscore(df))


# Define a threshold for Z-score beyond which data points are considered outliers
threshold = 4

# Filter out rows with any Z-score greater than the threshold
# Temporarily removing this 
outliers_removed_df =   features_all[(z_scores < threshold).all(axis=1)]

print(len(features_all))
print(len(outliers_removed_df))


## Standardizing the features

In [ ]:
## defining the global variables X and y
X = outliers_removed_df.drop(['serial_no','source'], axis = 1)
y = outliers_removed_df['source']


# Initialize the StandardScaler
scaler = StandardScaler()



# Apply standard scaling to the DataFrame
scaled_features = scaler.fit_transform(X)


# Access the mean and standard deviation for each feature
means = scaler.mean_
std_devs = scaler.scale_

# Create a DataFrame to display the means and standard deviations
scaler_params = pd.DataFrame({'Feature': X.columns, 'Mean': means, 'Std Dev': std_devs})
print(scaler_params)


# Create a new DataFrame with scaled features
X_scaled = pd.DataFrame(scaled_features, columns=X.columns)


#We are not standardizing at this stage. We will rather wait when the outlier are removed, then we will
## standardize and save the standard scaler parameters. 
X_scaled = X

X_scaled['serial_no'] = outliers_removed_df['serial_no'].values
X_scaled['source'] = outliers_removed_df['source'].values

## Merging the Metadata information with features

In [ ]:
# extracting the stored data
comcat_file_name = h5py.File("/data/whd01/yiyu_data/PNWML/comcat_waveforms.hdf5",'r')
exotic_file_name = h5py.File("/data/whd01/yiyu_data/PNWML/exotic_waveforms.hdf5",'r')
noise_file_name = h5py.File("/data/whd01/yiyu_data/PNWML/noise_waveforms.hdf5",'r')


# extracting the catalog
comcat_file_csv = pd.read_csv("/data/whd01/yiyu_data/PNWML/comcat_metadata.csv")
exotic_file_csv = pd.read_csv("/data/whd01/yiyu_data/PNWML/exotic_metadata.csv")
noise_file_csv = pd.read_csv("/data/whd01/yiyu_data/PNWML/noise_metadata.csv")



# extracting the metadata corresponding to individual events
cat_exp = comcat_file_csv[comcat_file_csv['source_type'] == 'explosion']
cat_eq = comcat_file_csv[comcat_file_csv['source_type'] == 'earthquake']
cat_no = noise_file_csv
cat_su = exotic_file_csv[exotic_file_csv['source_type'] == 'surface event']



# extracting the index 
ind_exp = X_scaled[X_scaled['source'] == 'explosion']['serial_no'].values
ind_eq = X_scaled[X_scaled['source'] == 'earthquake']['serial_no'].values
ind_no = X_scaled[X_scaled['source'] == 'noise']['serial_no'].values
ind_su = X_scaled[X_scaled['source'] == 'surface']['serial_no'].values


df_exp = X_scaled[X_scaled['source'] == 'explosion']
exp_df = cat_exp.iloc[ind_exp]
exp_df['serial_no'] = ind_exp


df_eq = X_scaled[X_scaled['source'] == 'earthquake']
eq_df = cat_eq.iloc[ind_eq]
eq_df['serial_no'] = ind_eq



df_no = X_scaled[X_scaled['source'] == 'noise']
no_df = cat_no.iloc[ind_no]
no_df['serial_no'] = ind_no



df_su = X_scaled[X_scaled['source'] == 'surface']
su_df = cat_su.iloc[ind_su]
su_df['serial_no'] = ind_su





new_exp = pd.merge(df_exp,exp_df, on = 'serial_no')
new_eq = pd.merge(df_eq,eq_df, on = 'serial_no')
new_su = pd.merge(df_su,su_df, on = 'serial_no')
new_no = pd.merge(df_no,no_df, on = 'serial_no')
new_no['event_id'] = np.array(['noise'+str(i) for i in np.arange(len(new_no))])



X_final = pd.concat([new_exp, new_eq, new_su, new_no])
y = ['explosion']*len(new_exp)+['earthquake']*len(new_eq)+['surface']*len(new_su)+['noise']*len(new_no)



### Adding Hour of the Day, Days of Week and Month of Year as additional features as they have shown to significantly increase the performance of the machine learning model

In [ ]:
# new_exp contains the features and the corresponding metadata information. 
datetimes = X_final['trace_start_time'].values

hour_of_day = []
days_of_week = []
month_of_year = []
for dt_str in tqdm(datetimes):
        
    # Parse the datetime string
        dt = datetime.strptime(dt_str, '%Y-%m-%dT%H:%M:%S.%fZ')
        hod = dt.hour - 8.  # converting to local time. 
        moy = dt.month
        
        
        days_of_week.append(dt.weekday())
        hour_of_day.append(hod)
        month_of_year.append(moy)
        
X_final['hour_of_day'] = hour_of_day
X_final['day_of_week'] = days_of_week
X_final['month_of_year'] = month_of_year


## Standardizing the features

In [ ]:
temp_X = X_final.iloc[:,0:int(np.where(X_final.columns == 'serial_no')[0])]
#temp_X = temp_X.assign(hod=X_final['hour_of_day'].values, dow=X_final['day_of_week'].values, moy=X_final['month_of_year'].values)



# Initialize the StandardScaler
scaler = StandardScaler()

# Apply standard scaling to the DataFrame
scaled_features = scaler.fit_transform(temp_X)

# Create a new DataFrame with scaled features
temp_X = pd.DataFrame(scaled_features, columns= temp_X.columns)



## Hyperparameter tuning of the random forest model for binary classification

In [ ]:
# Apply random undersampling using imbalanced-learn library
rus = RandomUnderSampler(sampling_strategy={'earthquake':3000, 'explosion':3000, 'surface':0, 'noise':0})
X_resampled, y_resampled = rus.fit_resample(temp_X, y)

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit the LabelEncoder on the text labels and transform them to numeric labels
y_num = label_encoder.fit_transform(y_resampled)


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_num, test_size=0.2, stratify = y_num)



# Define the hyperparameter grid for randomized search
param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False]
}

# Initialize the Random Forest model
rf_model = RandomForestClassifier(random_state=42)

# Initialize the RandomizedSearchCV with 5-fold cross-validation
random_search = RandomizedSearchCV(
    rf_model, param_distributions=param_dist, n_iter=50, scoring='f1_macro', cv=10, verbose=0, random_state=42, n_jobs=-1
)

# Perform randomized grid search cross-validation
random_search.fit(X_train, y_train)

# Print the best parameters and their corresponding accuracy score
print("Best Parameters:", random_search.best_params_)
print("Best Accuracy:", random_search.best_score_)

# Evaluate the best model on the test set
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = np.sum(y_pred == y_test) / len(y_test)
print("Test Accuracy:", accuracy)




## Performance with the best model

In [ ]:
## Performance with best model

plt.style.use('seaborn')
cm = confusion_matrix(y_test, y_pred)
plot_confusion_matrix(cm)


## Classification report with the best model

In [ ]:
# Calculate the classification report
report = classification_report(y_test, y_pred, output_dict=True)
plot_classification_report(report)

## Computing the feature importance

In [ ]:
num_iter = 10
f_imp = []

for i in tqdm(range(num_iter)):

    # Apply random undersampling using imbalanced-learn library
    rus = RandomUnderSampler(sampling_strategy={'earthquake':3000, 'explosion':3000, 'surface':0, 'noise':0})
    X_resampled, y_resampled = rus.fit_resample(temp_X, y)

    # Initialize the LabelEncoder
    label_encoder = LabelEncoder()

    # Fit the LabelEncoder on the text labels and transform them to numeric labels
    y_num = label_encoder.fit_transform(y_resampled)


    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_num, test_size=0.2, stratify = y_num)

    # Perform randomized grid search cross-validation
    best_model.fit(X_train, y_train)


    f_imp.append(best_model.feature_importances_)

In [ ]:
final_imp = np.mean(f_imp, axis = 0)
features = X_train.columns.values
feature_colors = ['darkred']*49+['darkblue']*(len(features)-49)  # red for physical and blue for tsfel features. 

## Removing the 0 from the starting of each feature name
## the second argument in split specifies the maxsplit, the amount of times the split will be performed. 

feature_names = np.hstack([features[0:49],[features[49:][i].split('_', 1)[1] for i in range(len(features[49:]))]])

In [ ]:
# Save the array to disk
np.save('../results/final_imp_binary.npy', final_imp)

## Plotting the most important features for binary discrimination

In [ ]:
# Sample feature importances and feature labels
feature_importances = final_imp
feature_labels = feature_names

# Sort feature importances and feature labels together
sorted_indices = sorted(range(len(feature_importances)), key=lambda k: feature_importances[k], reverse=True)
sorted_feature_importances = [feature_importances[i] for i in sorted_indices]
sorted_feature_labels = [feature_labels[i] for i in sorted_indices]
colors = [feature_colors[i] for i in sorted_indices]

# Plotting
plt.figure(figsize=(10, 10))
bars = plt.barh(sorted_feature_labels[0:50], sorted_feature_importances[0:50])

# Color bars to match yticklabels
for bar, color in zip(bars, colors):
    bar.set_color(color)

# Color yticklabels
for label, color in zip(plt.gca().get_yticklabels(), colors):
    label.set_color(color)

# Create legend handles and labels
legend_handles = [plt.Rectangle((0,0),1,1, color='darkblue', ec='black'), plt.Rectangle((0,0),1,1, color='darkred', ec='black')]
legend_labels = ['Tsfel Features', 'Physical Features']

plt.legend(legend_handles, legend_labels, title='Features', fontsize='large', title_fontsize='large', frameon=True, facecolor='white', edgecolor='black')

plt.xlabel('Feature Importance', fontsize=15)
plt.ylabel('Feature', fontsize=15)
plt.title('Top 50 Feature Importances', fontsize=15)
plt.gca().invert_yaxis()  # Invert y-axis to display highest importance at the top
plt.show()


## Plotting the distribution of Imp features

In [ ]:
no_df = features_noise_physical
eq_df = features_earthquake_physical
su_df = features_exotic_physical[features_exotic_physical['source'] == 'surface']
exp_df = features_explosion_physical


filtered_no = interquartile(no_df)
filtered_su = interquartile(su_df)
filtered_eq = interquartile(eq_df)
filtered_exp = interquartile(exp_df)


plt.style.use('default')
# Set rc parameters for font size
plt.rcParams['xtick.labelsize'] = 16  # Font size for xtick labels
plt.rcParams['ytick.labelsize'] = 16  # Font size for ytick labels

fig, ax = plt.subplots(nrows = 2, ncols = 2, figsize = [15,10])






feature = 'Kurto_0.1_1' ## The time taken to reach the maximum amplitude of Envelope
l1 = 0
l2 = 40
#all_df = features_all
#data_range = all_df[feature].values.ptp()
num_bins = 100 #int(np.ceil(data_range / 0.5))
#ax[0,0].hist(x= filtered_no[feature].values, bins= num_bins, density = True,
#          color =  '#d62728', alpha = 0.5,label = 'Noise')
#ax[0,0].hist(x= filtered_su[feature].values, bins= num_bins,density = True,    
#          color = '#2ca02c', alpha = 0.5, label = 'Surface')
ax[0,0].hist(x= filtered_eq[feature].values, bins= num_bins,density = True,  
               color = '#1f77b4', alpha = 0.5, label = 'Earthquake')
ax[0,0].hist(x= filtered_exp[feature].values, bins=num_bins,density = True,  
 color = '#9467bd', alpha = 0.5, label = 'Explosion')
ax[0,0].legend()
#ax[0,0].set_xscale('log')
#ax[0,0].set_yscale('log')
ax[0,0].set_xlim(l1,l2)
ax[0,0].set_title(feature, fontsize = 20)





feature = 'DistMaxMeanFreqDTF' ## The time taken to reach the maximum amplitude of Envelope
l1 = 0
l2 = 0.01
#all_df = features_all
#data_range = all_df[feature].values.ptp()
num_bins = 200 #int(np.ceil(data_range / 0.5))
#ax[0,1].hist(x= filtered_no[feature].values, bins= num_bins, density = True,
#          color =  '#d62728', alpha = 0.5,label = 'Noise')
#ax[0,1].hist(x= filtered_su[feature].values, bins= num_bins,density = True,    
#          color = '#2ca02c', alpha = 0.5, label = 'Surface')
ax[0,1].hist(x= filtered_eq[feature].values, bins= num_bins,density = True,  
               color = '#1f77b4', alpha = 0.5, label = 'Earthquake')
ax[0,1].hist(x= filtered_exp[feature].values, bins= num_bins,density = True,  
   color = '#9467bd', alpha = 0.5, label = 'Explosion')
ax[0,1].legend()
#ax[0,1].set_xscale('log')
#ax[0,1].set_yscale('log')
ax[0,1].set_xlim(l1,l2)
ax[0,1].set_title(feature, fontsize = 20)







feature = 'VarFFT' ## The Kurtosis of envelope
l1 = 0
l2 = 10

#all_df = features_all
#data_range = all_df[feature].values.ptp()
num_bins = 50 #int(np.ceil(data_range / 0.5))
#ax[1,0].hist(x= filtered_no[feature].values, bins= 400, density = True,
#          color =  '#d62728', alpha = 0.5,label = 'Noise')
#ax[1,0].hist(x= filtered_su[feature].values, bins= 50,density = True,    
#          color = '#2ca02c', alpha = 0.5, label = 'Surface')
ax[1,0].hist(x= filtered_eq[feature].values, bins= num_bins,density = True,  
               color = '#1f77b4', alpha = 0.5, label = 'Earthquake')
ax[1,0].hist(x= filtered_exp[feature].values, bins= num_bins,density = True,  
   color = '#9467bd', alpha = 0.5, label = 'Explosion')
ax[1,0].legend()
#ax[1,0].set_yscale('log')
#ax[1,0].set_xscale('log')
#ax[1,0].set_xlim(l1,l2)
ax[1,0].set_title(feature, fontsize = 20)




feature = 'FCentroid' ## ratio of Max over median of envelope
l1 = 0
l2 = 0.01

#all_df = features_all
#data_range = all_df[feature].values.ptp()
num_bins = 50 #int(np.ceil(data_range / 0.5))
#ax[1,1].hist(x= filtered_no[feature].values, bins= 500, density = True,
#          color =  '#d62728', alpha = 0.5,label = 'Noise')
#ax[1,1].hist(x= filtered_su[feature].values, bins= num_bins,density = True,    
#          color = '#2ca02c', alpha = 0.5, label = 'Surface')
ax[1,1].hist(x= filtered_eq[feature].values, bins= 50,density = True,  
               color = '#1f77b4', alpha = 0.5, label = 'Earthquake')
ax[1,1].hist(x= filtered_exp[feature].values, bins=num_bins,density = True,  
   color = '#9467bd', alpha = 0.5, label = 'Explosion')
ax[1,1].legend(loc = 'upper right')
ax[1,1].set_xscale('log')
#ax[1,1].set_yscale('log')
#ax[1,1].set_xlim(l1,l2)
ax[1,1].set_title(feature, fontsize = 20)

























"""


feature = 'KurtoEnv' ## ratio of Max over median of envelope


l1 = 0
l2 = 0.01

#all_df = features_all
#data_range = all_df[feature].values.ptp()
num_bins = 50 #int(np.ceil(data_range / 0.5))
#ax[1,1].hist(x= filtered_no[feature].values, bins= 500, density = True,
#          color =  '#d62728', alpha = 0.5,label = 'Noise')
#ax[1,1].hist(x= filtered_su[feature].values, bins= num_bins,density = True,    
#          color = '#2ca02c', alpha = 0.5, label = 'Surface')
ax[2,0].hist(x= filtered_eq[feature].values, bins= 50,density = True,  
               color = '#1f77b4', alpha = 0.5, label = 'Earthquake')
ax[2,0].hist(x= filtered_exp[feature].values, bins=num_bins,density = True,  
   color = '#9467bd', alpha = 0.5, label = 'Explosion')

ax[2,0].legend(loc = 'upper right')
ax[2,0].set_xscale('log')
#ax[1,1].set_yscale('log')
#ax[1,1].set_xlim(l1,l2)


ax[2,0].set_title(feature, fontsize = 20)




feature = 'DistQ3Q1DFT' ## ratio of Max over median of envelope


l1 = 0
l2 = 0.01

#all_df = features_all
#data_range = all_df[feature].values.ptp()
num_bins = 50 #int(np.ceil(data_range / 0.5))
#ax[1,1].hist(x= filtered_no[feature].values, bins= 500, density = True,
#          color =  '#d62728', alpha = 0.5,label = 'Noise')
#ax[1,1].hist(x= filtered_su[feature].values, bins= num_bins,density = True,    
#          color = '#2ca02c', alpha = 0.5, label = 'Surface')
ax[2,1].hist(x= filtered_eq[feature].values, bins= 50,density = True,  
               color = '#1f77b4', alpha = 0.5, label = 'Earthquake')
ax[2,1].hist(x= filtered_exp[feature].values, bins=num_bins,density = True,  
   color = '#9467bd', alpha = 0.5, label = 'Explosion')

ax[2,1].legend(loc = 'upper right')
#ax[2,1].set_xscale('log')
#ax[1,1].set_yscale('log')
#ax[1,1].set_xlim(l1,l2)


ax[2,1].set_title(feature, fontsize = 20)






fig.supylabel('Density', fontsize = 20)
fig.supxlabel('Value of Features', fontsize = 20)
fig.tight_layout()


"""

### Plotting the most important tsfel features

In [ ]:
no_df = features_noise_tsfel
eq_df = features_earthquake_tsfel
su_df = features_exotic_tsfel[features_exotic_tsfel['source'] == 'surface']
exp_df = features_explosion_tsfel

filtered_no = interquartile(no_df)
filtered_su = interquartile(su_df)
filtered_eq = interquartile(eq_df)
filtered_exp = interquartile(exp_df)


# Define a function for histogram plotting
def plot_histogram(ax, feature_data, label, color, num_bins, title):
    ax.hist(x=feature_data, bins=num_bins, density=True, color=color, alpha=0.5, label=label)
    ax.legend()
    ax.set_title(title, fontsize=20)

# Define common parameters
num_bins = 50
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=[15, 10])

# Define features and labels
features = {
    '0_Max power spectrum': [filtered_eq, filtered_exp],
    '0_Spectral entropy': [filtered_eq, filtered_exp],
    '0_Histogram_2': [filtered_eq, filtered_exp],
    '0_FFT mean coefficient_9': [filtered_eq, filtered_exp]
}
labels = ['Earthquake', 'Explosion']
colors = ['#1f77b4', '#9467bd']

# Iterate over features and plot histograms
for idx, (feature, (filtered_eq, filtered_exp)) in enumerate(features.items()):
    row = idx // 2
    col = idx % 2
    plot_histogram(ax[row, col], filtered_eq[feature].values, labels[0], colors[0], num_bins, feature.split('_')[1])
    plot_histogram(ax[row, col], filtered_exp[feature].values, labels[1], colors[1], num_bins, feature.split('_')[1])

fig.supylabel('Density', fontsize=20)
fig.supxlabel('Value of Features', fontsize=20)
fig.tight_layout()
plt.show()


## Selecting 5000 random events per class for training and testing on all the remaining data

In [ ]:
## extracting metadata information for each kind of source along with features
a_eq = X_final[X_final['source_type_pnsn_label'] == 'eq']
## removing the ambiguous events
a_eq = a_eq[a_eq['source_type'] != 'explosion']

a_px = X_final[X_final['source_type_pnsn_label'] == 'px']
# removing the ambiguous events. 
a_px = a_px[a_px['source_type'] != 'earthquake']

a_su = X_final[X_final['source_type'] == 'surface event']
a_no = X_final[X_final['source_type'] == 'noise']



## extracting the event ids corresponding to each catalog
eq_ids = np.unique(a_eq['event_id'].values)
su_ids = np.unique(a_su['event_id'].values)
no_ids = np.unique(a_no['event_id'].values)
px_ids = np.unique(a_px['event_id'].values)


## defining the events for training and testing in 70:30 ratio


"""

Modifying this section a bit, for a fair comparison to deep neural network. 


train_eq = eq_ids[0:int(0.7*len(eq_ids))]
train_px = px_ids[0:int(0.7*len(px_ids))]
train_su = su_ids[0:int(0.7*len(su_ids))]
train_no = no_ids[0:int(0.7*len(no_ids))]


test_eq = eq_ids[int(0.7*len(eq_ids)):len(eq_ids)]
test_px = px_ids[int(0.7*len(px_ids)):len(px_ids)]
test_su = su_ids[int(0.7*len(su_ids)):len(su_ids)]
test_no = no_ids[int(0.7*len(no_ids)):len(no_ids)]


"""


# Set the seed for reproducibility
np.random.seed(42)



## randomizing along the time. 
r1 = np.random.randint(0, len(eq_ids), 5000)
train_eq = eq_ids[r1]

## randomizing along the time. 
r2 = np.random.randint(0, len(px_ids), 5000)
train_px = px_ids[r2]

## randomizing along the time. 
r3 = np.random.randint(0, len(su_ids), 5000)
train_su = su_ids[r3]

## randomizing along the time
r4 = np.random.randint(0, len(no_ids), 5000)
train_no = no_ids[r4]



mask_eq = np.ones(eq_ids.shape, dtype = bool)
mask_eq[r1] = False

mask_px = np.ones(px_ids.shape, dtype = bool)
mask_px[r2] = False

mask_su = np.ones(su_ids.shape, dtype = bool)
mask_su[r3] = False

mask_no = np.ones(no_ids.shape, dtype = bool)
mask_no[r4] = False

test_eq = eq_ids[mask_eq]
test_px = px_ids[mask_px]
test_su = su_ids[mask_su]
test_no = no_ids[mask_no]



# concatenating training ids
all_train_ids = np.concatenate([train_eq,train_px])

# concatenating testing ids
all_test_ids = np.concatenate([test_eq,test_px,])

# allocating event id as index
X_final.index = X_final['event_id'].values



In [ ]:
# extracting training and testing values
X_train = X_final.loc[all_train_ids]
X_test = X_final.loc[all_test_ids]



Y_train = X_train['source_type'].values
Y_test = X_test['source_type'].values

## Performance with manual features added

In [ ]:
# Case 1: without adding anything manual
## Check the performance 
x_train = X_train.iloc[:, 0:int(np.where(X_train.columns == 'serial_no')[0])]
x_train = x_train.assign(hod=X_train['hour_of_day'].values, dow=X_train['day_of_week'].values, moy=X_train['month_of_year'].values)

x_test = X_test.iloc[:, 0:int(np.where(X_train.columns == 'serial_no')[0])]
x_test = x_test.assign(hod=X_test['hour_of_day'].values, dow=X_test['day_of_week'].values, moy=X_test['month_of_year'].values)



## Training on 5000 traces per class and testing on all the remaining

In [ ]:
# initiating a random undersampler
rus = RandomUnderSampler(sampling_strategy={'earthquake':5000, 'explosion':5000}, random_state = 42)

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Converting the textual labels into numerical labels
y_num_test = label_encoder.fit_transform(Y_test)


# randomly taking 5000 samples per class from the training dataset
X_resampled, y_resampled = rus.fit_resample(x_train, Y_train)


# Fit the LabelEncoder on the text labels and transform them to numeric labels
y_num_res = label_encoder.fit_transform(y_resampled)



best_model.class_weight  = None
best_model.fit(X_resampled, y_num_res)




## Trace wise performance

In [ ]:
## Trace wise performance 

y_pred = best_model.predict(x_test)
plt.style.use('seaborn')
trace_cm_phy_tsf_man = confusion_matrix(y_num_test, y_pred)
plot_confusion_matrix(trace_cm_phy_tsf_man)

# Calculate the classification report
trace_report_phy_tsf_man = classification_report(y_num_test, y_pred, output_dict=True)
plot_classification_report(trace_report_phy_tsf_man)

## Event Wise performance

In [ ]:
probs_all = best_model.predict_proba(x_test)

X_test['labelled'] = y_num_test
X_test['classified'] = y_pred
X_test['eq_probability'] = probs_all[:,0]
X_test['px_probability'] = probs_all[:,1]



mean_labels = X_test.groupby('event_id').mean()['labelled'].values
mean_ids = X_test.groupby('event_id').mean().index.values



mean_eq_prob = X_test.groupby('event_id').mean()['eq_probability'].values
mean_px_prob = X_test.groupby('event_id').mean()['px_probability'].values




## Assigning an event class based on the maximum average probability across the stations. 
temp_class = np.argmax(np.vstack([mean_eq_prob, mean_px_prob]), axis = 0)
## Computing the maximum averaged probability. 
temp_probs = np.max(np.vstack([mean_eq_prob, mean_px_prob]), axis = 0)



cf_events_phy_tsf_man = confusion_matrix(mean_labels, temp_class)
plot_confusion_matrix(cf_events_phy_tsf_man)



# Calculate the classification report
report_event_phy_tsf_man = classification_report(mean_labels, temp_class, output_dict=True)
plot_classification_report(report_event_phy_tsf_man)

## Analysis of misclassified events

In [ ]:
# probabilities corresponding to misclassified events
misclassified_probs = temp_probs[np.where(mean_labels != temp_class)]

# classification provided to misclassified events
misclassified_class = temp_class[np.where(mean_labels != temp_class)]

# labels provided to misclassified events. 
misclassified_labels = mean_labels[np.where(mean_labels != temp_class)]

# event ids provided to misclassified events. 
misclassified_ids = mean_ids[np.where(mean_labels != temp_class)]

# event ids from misclassified events where mean probabilities exceed 0.8
misclassified_high_prob = misclassified_ids[np.where(misclassified_probs > 0.8)]


# Dataframe of misclassified events
misclassified_X = X_test.loc[misclassified_high_prob]



## New objectives

- download more stations data for each of the misclassified events. 
- extract features
- classify them and see if they still going to be misclassified. 

In [ ]:
mis_ids = X_test.loc[misclassified_ids].groupby(level=0).size()
#indices_with_value_one = misclassified_ids[misclassified_ids == 1].index
single_station_event = mis_ids[mis_ids < 3].index.values

## Downloading more station waveforms for each of the single event. 

In [ ]:
X_test.loc[single_station_event]

In [ ]:
starttimes  = X_test.loc[single_station_event]['trace_start_time'].values
source_lats = X_test.loc[single_station_event]['source_latitude_deg'].values
source_lons = X_test.loc[single_station_event]['source_longitude_deg'].values
event_ids = X_test.loc[single_station_event].index.values


In [ ]:
client = Client("IRIS")
for i in tqdm(range(len(event_ids))):
    
    try:


        stns = client.get_stations(channel = '*HZ', 
                                starttime=obspy.UTCDateTime(starttimes[i]),
                                endtime= obspy.UTCDateTime(starttimes[i])+150,  
                                latitude= source_lats[i], 
                                longitude = source_lons[i], 
                                minradius= 0, 
                                maxradius= 0.3, 
                                level = 'response'
                             )
        
        for network in stns:
            for station in network:
                for channel in station:
                    st = client.get_waveforms(starttime = obspy.UTCDateTime(starttimes[i]), endtime = obspy.UTCDateTime(starttimes[i]) + 150, network = network.code, station = station.code, channel = channel.code, location = '*')
                    st.write('../data/Binary_Classification/'+event_ids[i]+'_'+network.code+'.'+station.code+'.'+channel.code+'.'+str(starttime)+'.mseed')
                    
                    
    except:
        pass


## Results after including more stations per waveform. 

## Data processing

In [ ]:
# location of the waveform files in the disk. 
files = glob('../data/binary_classification_waveforms/*.mseed')



### Extracting data from each waveform

In [ ]:
data = []
event_ids = []
for file in tqdm(files):
    st = obspy.read(file)
    st.resample(100)
    if len(st[0].data) == 15000:
        event_ids.append(file.split('/')[-1])   ### .split('_')[0])
        
        # depending on what features the model was trained on we select the appropriate window. 
        # The current saved model was trained on the 150s window waveforms (P-50, P+100). So we will load those waveforms
        data.append(st[0].data)

In [ ]:
# stacking the data
data = np.stack(data)

# applying 10% cosine taper before filtering
tapered = apply_cosine_taper(data)

# defining the parameters of the filter. 
lowcut = 1
highcut = 10
num_corners = 4
fs = 100


# Applying bandpass filters
filtered = butterworth_filter(tapered, lowcut, highcut, fs, num_corners, filter_type='bandpass')

# Applying the normalization. 
norm = filtered/np.max(abs(np.stack(filtered)), axis = 1)[:, np.newaxis]

## Extracting tsfel features

In [ ]:
code_sttime = time.time()
cfg_file = tsfel.get_features_by_domain()
# Extract features for surface event
features_bin_tsfel = pd.DataFrame([])
for i in tqdm(range(len(norm))):

        df = time_series_features_extractor(cfg_file, norm[i],  fs=100)
        df['event_id'] = event_ids[i]
        features_bin_tsfel = pd.concat([features_bin_tsfel,df])

print(time.time() - code_sttime)

In [ ]:
a = time_series_features_extractor(cfg_file, norm[i],  fs=100)

In [ ]:
a

## Extracting physical features

In [ ]:
code_sttime = time.time()
features_bin_physical = pd.DataFrame([])
for i in tqdm(range(len(norm))):
    try: 
        df = seis_feature.FeatureCalculator(norm[i]).compute_features()
        df['serial_no'] = i
        df['event_id'] = event_ids[i]
        #df['trace_id'] = trids[i]
        features_bin_physical = pd.concat([features_bin_physical, df])
    except:
        pass
    
print(time.time() - code_sttime)

In [ ]:
features_bin_physical

In [ ]:
features_bin_tsfel

In [ ]:
## Removing the correlated features
features_bin_tsfel.drop(corr_features, axis=1, inplace=True, errors = 'ignore')
final_features_bin = pd.concat([features_bin_tsfel, features_bin_physical], axis = 1)
columns_to_select = scaler_params['Feature'].values
features_bin = final_features_bin.loc[:, columns_to_select]

In [ ]:
#for i in range(len(features_bin.columns)):
#    features_bin.iloc[:,i] = (features_bin.iloc[:,i]-scaler_params.iloc[i,1])/(scaler_params.iloc[i,2])
    


In [ ]:
hod_bin = []
dow_bin = []
moy_bin = []

for i in range(len(event_ids)):
    value = X_test.loc[event_ids[i]]['hour_of_day']
    hod_bin.append(value if isinstance(value, np.float64) else value.values[0])
    
    value = X_test.loc[event_ids[i]]['day_of_week']
    dow_bin.append(value if isinstance(value, np.int64) else value.values[0])
    
    value = X_test.loc[event_ids[i]]['month_of_year']
    moy_bin.append(value if isinstance(value, np.int64) else value.values[0])


In [ ]:
features_bin['hour_of_day'] = hod_bin
features_bin['day_of_week'] = dow_bin
features_bin['month_of_year'] = moy_bin

In [ ]:
features_bin


In [ ]:
## Trace wise performance 

y_pred_bin = best_model.predict(features_bin)

In [ ]:
features_bin['Eq_prob'] = best_model.predict_proba(features_bin.iloc[:, 0:317])[:,0]
features_bin['Exp_prob'] = best_model.predict_proba(features_bin.iloc[:, 0:317])[:,1]

In [ ]:
features_bin['event_id'] = event_ids

In [ ]:
event_labels_bin = np.argmax(features_bin.groupby('event_id').mean().iloc[:, 318:320].values, axis = 1)

In [ ]:
event_ids_mean = features_bin.groupby('event_id').mean().index.values

In [ ]:
features_bin[features_bin['event_id'] == event_ids_mean[4]]

In [ ]:
test_overall = pd.DataFrame([])
test_overall['mean_labels'] = mean_labels
test_overall['mean_ids'] = mean_ids
test_overall['assigned_class'] = temp_class
test_overall.index = mean_ids

In [ ]:
for i in range(len(event_ids_mean)):
    test_overall.loc[event_ids_mean[i], 'assigned_class'] = event_labels_bin[i]


In [ ]:
cf_bin = confusion_matrix(test_overall['mean_labels'].values, test_overall['assigned_class'].values)
#cf_norm = cf_events/np.sum(cf_events, axis = 1, keepdims = True)
class_labels = ['earthquake','explosion']

plt.figure(figsize = [8, 6])
ax = sns.heatmap(cf_bin, annot = True, cmap='Blues', xticklabels = class_labels, yticklabels = class_labels,  fmt="1.0f", annot_kws=annot_kws)
# Set tick label font size
ax.set_xticklabels(class_labels, fontsize=15)
ax.set_yticklabels(class_labels, fontsize=15)

ax.set_xlabel('Predicted', fontsize = 15)
ax.set_ylabel('Labeled', fontsize = 15)
ax.set_title('Total Events: '+str(len(mean_labels)), fontsize = 20)
plt.tight_layout()


In [ ]:
# Calculate the classification report
report_bin = classification_report(test_overall['mean_labels'].values, test_overall['assigned_class'].values, output_dict=True)
labels = ['Precision', 'Recall', 'F1-Score']

# Set a pleasing style
sns.set_style("whitegrid")

# Create a figure and axes for the heatmap
plt.figure(figsize = [8,6])
ax = sns.heatmap(pd.DataFrame(report_bin).iloc[:3, :2], annot=True, cmap='Blues', yticklabels=labels, xticklabels = class_labels,  vmin=0.8, vmax=1,  annot_kws=annot_kws)

# Set tick label font size
ax.set_xticklabels(class_labels, fontsize=15)
ax.set_yticklabels(labels, fontsize=15)

# Set labels and title
ax.set_xlabel('Metrics', fontsize=15)
ax.set_ylabel('Classes', fontsize=15)
ax.set_title('Classification Report', fontsize=18)

# Create a colorbar
#cbar = ax.collections[0].colorbar
#cbar.set_ticks([0.5, 1])  # Set custom tick locations
#cbar.set_ticklabels(['0', '0.5', '1'])  # Set custom tick labels

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()


### Lets try to understand why including more stations is improving the results. 

In [ ]:
## Collecting and storing event ids
files = glob('../data/binary_classification_waveforms/*.mseed')
station_ids = []
for file in tqdm(files):
    st = obspy.read(file)
    st.resample(100)
    if len(st[0].data) == 15000:
        station_ids.append(st[0].id)

In [ ]:
k = 7

## Grabbing the data from the stations
sample_data = norm[np.where(np.array(event_ids) == event_ids_mean[k])[0]]

# Grabbing the station ids 
sample_station_ids = np.array(station_ids)[np.where(np.array(event_ids) == event_ids_mean[k])[0]]

# Grabbing the results
sample_df_here = features_bin[features_bin['event_id'] == event_ids_mean[k]]
sample_eq_probs = sample_df_here['Eq_prob'].values
sample_exp_probs = sample_df_here['Exp_prob'].values


# Grabbing the original labels
original_label = X_test.loc[event_ids_mean[4]]['labelled']

time_here = np.linspace(0, 150, 15000)

# Create a figure and subplots
fig, axs = plt.subplots(nrows=len(sample_data), ncols=1, figsize=(12, 3*len(sample_data)), sharex = True)  # Adjust figsize as needed




# Plot data on each subplot
for i in range(len(sample_data)):
    axs[i].plot(time_here, sample_data[i], lw=0.5)
    #axs[i].set_xlim(40, 80)
    axs[i].set_title(sample_station_ids[i]+' Eq prob:'+str(np.around(sample_eq_probs[i],2))+' Exp prob:'+str(np.around(sample_exp_probs[i], 2)))  # Set title for each subplot
    #axs[i].set_xlabel('Time(s)')  # Set x-axis label (optional)
    axs[i].set_ylabel('Amplitude')  # Set y-axis label (optional)

    
    
# Add common labels to the entire figure
fig.suptitle('PNSN Label: '+str(original_label), y = 0.99)  # Title for the entire figure
fig.supylabel('Common Y-Axis Label')  # Common y-axis label
fig.supxlabel('Time(s)')  # Common x-axis label

# Adjust layout to prevent overlap
plt.tight_layout()

# Display the combined figure with subplots
plt.show()


In [ ]:
features = scaler_params['Feature'].values
#for i in range(len(scaler_params)):
#    X_test.loc[:,features[i]] = X_test.loc[:,features[i]]*(scaler_params.iloc[i,2]) + scaler_params.iloc[i,1]
    
    
misclassified_df = X_test.loc[misclassified_ids]
misclassified_eq = misclassified_df[misclassified_df['classified'] == 1]
misclassified_exp = misclassified_df[misclassified_df['classified'] == 0]

In [ ]:
new_eq = features_earthquake_physical
new_exp = features_explosion_physical


fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = [20,15])

feature = 'DistMaxMedianFreqDTF'


l1 = 0
l2 = 0.1

all_df = features_all
data_range = all_df[feature].values.ptp()
num_bins = int((l2 - l1)/ 0.0005)
#sns.histplot(x= no_df[feature].values, bins= num_bins, kde=False, ax = ax[0,0], 
#             multiple='stack', stat = 'percent', color = 'green', alpha = 0.5,label = 'Noise')
#sns.histplot(x= su_df[feature].values, bins= num_bins, kde=False, ax = ax[0,0],   
#             multiple='stack', stat = 'percent', color = 'red', alpha = 0.5, label = 'Surface')
sns.histplot(x= features_earthquake_physical[feature].values, bins= num_bins, kde=False, ax = ax[0,0],  
             multiple='stack', stat = 'percent', color =  '#1f77b4', alpha = 0.3, label = 'Earthquake')
sns.histplot(x= features_explosion_physical[feature].values, bins=num_bins, kde=False, ax = ax[0,0],  
             multiple='stack', stat = 'percent', color = '#9467bd', alpha = 0.3, label = 'Explosion')

X_miss_earthquake = misclassified_eq[misclassified_eq['px_probability'] > 0.8]
X_miss_explosion = misclassified_exp[misclassified_exp['eq_probability'] > 0.8]



ax[0,0].scatter(X_miss_earthquake[feature].values, np.ones(len(X_miss_earthquake))*10, s = 300, ec = 'k', marker = '^',c = X_miss_earthquake['px_probability'], cmap = 'Reds',label = 'Miss. Eq')
ax[0,0].scatter(X_miss_explosion[feature].values, np.ones(len(X_miss_explosion))*5, s = 300, ec = 'k', marker = '*',c = X_miss_explosion['eq_probability'], cmap = 'Reds',label = 'Miss. Exp')


#ax[0,0].scatter(X_miss_explosion[feature].values[np.where(probs_miss_explosion > 0.9)[0]], np.ones(len(X_miss_explosion.iloc[np.where(probs_miss_explosion > 0.9)[0]]))*0.5, s = 50, marker = 'o',ec = 'k',c = probs_miss_explosion[np.where(probs_miss_explosion > 0.9)[0]], cmap = 'magma',label = 'Miss. exp')
ax[0,0].legend()
#ax[0,0].set_xscale('log')
ax[0,0].set_xlim(l1,0.03)




ax[0,0].set_title(feature, fontsize = 20)





feature = 'DistMaxMeanFreqDTF'

l1 = 0
l2 = 0.1

all_df = features_all
data_range = all_df[feature].values.ptp()
num_bins = int((l2 - l1)/ 0.0005)
#sns.histplot(x= no_df[feature].values, bins= num_bins, kde=False, ax = ax[0,0], 
#             multiple='stack', stat = 'percent', color = 'green', alpha = 0.5,label = 'Noise')
#sns.histplot(x= su_df[feature].values, bins= num_bins, kde=False, ax = ax[0,0],   
#             multiple='stack', stat = 'percent', color = 'red', alpha = 0.5, label = 'Surface')
sns.histplot(x= new_eq[feature].values, bins= num_bins, kde=False, ax = ax[0,1],  
             multiple='stack',  stat = 'percent', color =  '#1f77b4', alpha = 0.3, label = 'Earthquake')
sns.histplot(x= new_exp[feature].values, bins=num_bins, kde=False, ax = ax[0,1],  
             multiple='stack', stat = 'percent', color = '#9467bd', alpha = 0.3, label = 'Explosion')




ax[0,1].scatter(X_miss_earthquake[feature].values, np.ones(len(X_miss_earthquake))*10,ec = 'k', s = 300, marker = '^',c = X_miss_earthquake['px_probability'], cmap = 'Reds',label = 'Miss. Eq')
ax[0,1].scatter(X_miss_explosion[feature].values, np.ones(len(X_miss_explosion))*5,ec = 'k',  s = 300, marker = '*',c = X_miss_explosion['eq_probability'], cmap = 'Reds',label = 'Miss. Exp')




ax[0,1].legend()
#ax[0,0].set_xscale('log')
ax[0,1].set_xlim(l1,0.03)


ax[0,1].set_title(feature, fontsize = 20)






feature = 'E4FFT'

l1 = -0.35
l2 = -0.1

all_df = features_all
data_range = all_df[feature].values.ptp()
num_bins = int(data_range/ 0.05)
#sns.histplot(x= no_df[feature].values, bins= num_bins, kde=False, ax = ax[0,0], 
#             multiple='stack', stat = 'percent', color = 'green', alpha = 0.5,label = 'Noise')
#sns.histplot(x= su_df[feature].values, bins= num_bins, kde=False, ax = ax[0,0],   
#             multiple='stack', stat = 'percent', color = 'red', alpha = 0.5, label = 'Surface')
sns.histplot(x= new_eq[feature].values, bins= 2000, kde=False, ax = ax[0,2],  
             multiple='stack',  stat = 'percent', color =  '#1f77b4', alpha = 0.3, label = 'Earthquake')
sns.histplot(x= new_exp[feature].values, bins= 10, kde=False, ax = ax[0,2],  
             multiple='stack', stat = 'percent', color = '#9467bd', alpha = 0.3, label = 'Explosion')


ax[0,2].scatter(X_miss_earthquake[feature].values, np.ones(len(X_miss_earthquake))*40,ec = 'k',  s = 300, marker = '^',c = X_miss_earthquake['px_probability'], cmap = 'Reds',label = 'Miss. Eq')
ax[0,2].scatter(X_miss_explosion[feature].values, np.ones(len(X_miss_explosion))*20, ec = 'k', s = 300, marker = '*',c = X_miss_explosion['eq_probability'], cmap = 'Reds',label = 'Miss. Exp')
ax[0,2].legend()
#ax[0,2].set_xscale('log')
ax[0,2].set_xlim(0,0.05)


ax[0,2].set_title(feature, fontsize = 20)




feature = 'Gamma1'

l1 = 0
l2 = 0.1

all_df = features_all
data_range = all_df[feature].values.ptp()
num_bins = int((l2 - l1)/ 0.0005)
#sns.histplot(x= no_df[feature].values, bins= num_bins, kde=False, ax = ax[0,0], 
#             multiple='stack', stat = 'percent', color = 'green', alpha = 0.5,label = 'Noise')
#sns.histplot(x= su_df[feature].values, bins= num_bins, kde=False, ax = ax[0,0],   
#             multiple='stack', stat = 'percent', color = 'red', alpha = 0.5, label = 'Surface')
sns.histplot(x= new_eq[feature].values, bins= num_bins, kde=False, ax = ax[1,0],  
             multiple='stack',  stat = 'percent', color =  '#1f77b4', alpha = 0.3, label = 'Earthquake')
sns.histplot(x= new_exp[feature].values, bins=num_bins, kde=False, ax = ax[1,0],  
             multiple='stack', stat = 'percent', color = '#9467bd', alpha = 0.3, label = 'Explosion')
#ax[1,0].scatter(X_miss_earthquake[feature].values[np.where(probs_miss_earthquake > 0.9)[0]], np.ones(len(X_miss_earthquake.iloc[np.where(probs_miss_earthquake > 0.9)[0]]))*1, s = 50, marker = '^',ec = 'k',c = probs_miss_earthquake[np.where(probs_miss_earthquake > 0.9)[0]], cmap = 'magma',label = 'Miss. eqs')
#ax[1,0].scatter(X_miss_explosion[feature].values[np.where(probs_miss_explosion > 0.9)[0]], np.ones(len(X_miss_explosion.iloc[np.where(probs_miss_explosion > 0.9)[0]]))*0.5, s = 50, marker = 'o',ec = 'k',c = probs_miss_explosion[np.where(probs_miss_explosion > 0.9)[0]], cmap = 'magma',label = 'Miss. exp')

ax[1,0].scatter(X_miss_earthquake[feature].values, np.ones(len(X_miss_earthquake))*1, s = 300, ec = 'k', marker = '^',c = X_miss_earthquake['px_probability'], cmap = 'Reds',label = 'Miss. Eq')
ax[1,0].scatter(X_miss_explosion[feature].values, np.ones(len(X_miss_explosion))*0.5, s = 300, ec = 'k', marker = '*',c = X_miss_explosion['eq_probability'], cmap = 'Reds',label = 'Miss. Exp')

ax[1,0].legend()
#ax[0,0].set_xscale('log')
#ax[0,0].set_xlim(l1,0.02)


ax[1,0].set_title(feature, fontsize = 20)





feature = 'Gamma2'

l1 = 0
l2 = 0.1

all_df = features_all
data_range = all_df[feature].values.ptp()
num_bins = int((l2 - l1)/ 0.0005)
#sns.histplot(x= no_df[feature].values, bins= num_bins, kde=False, ax = ax[0,0], 
#             multiple='stack', stat = 'percent', color = 'green', alpha = 0.5,label = 'Noise')
#sns.histplot(x= su_df[feature].values, bins= num_bins, kde=False, ax = ax[0,0],   
#             multiple='stack', stat = 'percent', color = 'red', alpha = 0.5, label = 'Surface')
sns.histplot(x= new_eq[feature].values, bins= num_bins, kde=False, ax = ax[1,1],  
             multiple='stack', stat = 'percent', color =  '#1f77b4', alpha = 0.5, label = 'Earthquake')
sns.histplot(x= new_exp[feature].values, bins=num_bins, kde=False, ax = ax[1,1],  
             multiple='stack', stat = 'percent', color = '#9467bd', alpha = 0.5, label = 'Explosion')


ax[1,1].scatter(X_miss_earthquake[feature].values, np.ones(len(X_miss_earthquake))*1, s = 300, ec = 'k', marker = '^',c = X_miss_earthquake['px_probability'], cmap = 'Reds',label = 'Miss. Eq')
ax[1,1].scatter(X_miss_explosion[feature].values, np.ones(len(X_miss_explosion))*0.5, s = 300, ec = 'k', marker = '*',c = X_miss_explosion['eq_probability'], cmap = 'Reds',label = 'Miss. Exp')


ax[1,1].legend()
#ax[0,0].set_xscale('log')
#ax[0,0].set_xlim(l1,0.02)


ax[1,1].set_title(feature, fontsize = 20)






feature = 'Fquart1'

l1 = 0
l2 = 0.1

all_df = features_all
data_range = all_df[feature].values.ptp()
num_bins = int((l2 - l1)/ 0.0005)
#sns.histplot(x= no_df[feature].values, bins= num_bins, kde=False, ax = ax[0,0], 
#             multiple='stack', stat = 'percent', color = 'green', alpha = 0.5,label = 'Noise')
#sns.histplot(x= su_df[feature].values, bins= num_bins, kde=False, ax = ax[0,0],   
#             multiple='stack', stat = 'percent', color = 'red', alpha = 0.5, label = 'Surface')
sns.histplot(x= new_eq[feature].values, bins= num_bins, kde=False, ax = ax[1,2],  
             multiple='stack',  stat = 'percent', color =  '#1f77b4', alpha = 0.3, label = 'Earthquake')
sns.histplot(x= new_exp[feature].values, bins=num_bins, kde=False, ax = ax[1,2],  
             multiple='stack', stat = 'percent', color = '#9467bd', alpha = 0.3, label = 'Explosion')


ax[1,2].scatter(X_miss_earthquake[feature].values, np.ones(len(X_miss_earthquake))*1, s = 300, ec = 'k', marker = '^',c = X_miss_earthquake['px_probability'], cmap = 'Reds',label = 'Miss. Eq')
ax[1,2].scatter(X_miss_explosion[feature].values, np.ones(len(X_miss_explosion))*0.5, s = 300, ec = 'k', marker = '*',c = X_miss_explosion['eq_probability'], cmap = 'Reds',label = 'Miss. Exp')
ax[1,2].legend()
#ax[0,0].set_xscale('log')
#ax[0,0].set_xlim(l1,0.02)


ax[1,2].set_title(feature, fontsize = 20)






new_eq = features_earthquake_tsfel
new_exp = features_explosion_tsfel












feature = '0_Max power spectrum'

l1 = 0
l2 = 0.1

all_df = features_all
data_range = all_df[feature].values.ptp()
num_bins = int((l2 - l1)/ 0.0005)
#sns.histplot(x= no_df[feature].values, bins= num_bins, kde=False, ax = ax[0,0], 
#             multiple='stack', stat = 'percent', color = 'green', alpha = 0.5,label = 'Noise')
#sns.histplot(x= su_df[feature].values, bins= num_bins, kde=False, ax = ax[0,0],   
#             multiple='stack', stat = 'percent', color = 'red', alpha = 0.5, label = 'Surface')
sns.histplot(x= new_eq[feature].values, bins= num_bins, kde=False, ax = ax[2,0],  
             multiple='stack', stat = 'percent',  color =  '#1f77b4', alpha = 0.3, label = 'Earthquake')
sns.histplot(x= new_exp[feature].values, bins=num_bins, kde=False, ax = ax[2,0],  
             multiple='stack', stat = 'percent', color = '#9467bd', alpha = 0.3, label = 'Explosion')



ax[2,0].scatter(X_miss_earthquake[feature].values, np.ones(len(X_miss_earthquake))*10, s = 300,ec = 'k', marker = '^',c = X_miss_earthquake['px_probability'], cmap = 'Reds',label = 'Miss. Eq')
ax[2,0].scatter(X_miss_explosion[feature].values, np.ones(len(X_miss_explosion))*5, s = 300, ec = 'k', marker = '*',c = X_miss_explosion['eq_probability'], cmap = 'Reds',label = 'Miss. Exp')
ax[2,0].legend()
#ax[0,0].set_xscale('log')
ax[2,0].set_xlim(0,30)


ax[2,0].set_title(feature, fontsize = 20)






feature = '0_Spectral entropy'

l1 = 0
l2 = 0.1

all_df = features_all
data_range = all_df[feature].values.ptp()
num_bins = int((l2 - l1)/ 0.0005)
#sns.histplot(x= no_df[feature].values, bins= num_bins, kde=False, ax = ax[0,0], 
#             multiple='stack', stat = 'percent', color = 'green', alpha = 0.5,label = 'Noise')
#sns.histplot(x= su_df[feature].values, bins= num_bins, kde=False, ax = ax[0,0],   
#             multiple='stack', stat = 'percent', color = 'red', alpha = 0.5, label = 'Surface')
sns.histplot(x= new_eq[feature].values, bins= num_bins, kde=False, ax = ax[2,1],  
             multiple='stack', stat = 'percent',  color =  '#1f77b4', alpha = 0.3, label = 'Earthquake')
sns.histplot(x= new_exp[feature].values, bins=num_bins, kde=False, ax = ax[2,1],  
             multiple='stack', stat = 'percent',  color = '#9467bd', alpha = 0.3, label = 'Explosion')


ax[2,1].scatter(X_miss_earthquake[feature].values, np.ones(len(X_miss_earthquake))*4, s = 300, marker = '^',ec = 'k',c = X_miss_earthquake['px_probability'], cmap = 'Reds',label = 'Miss. Eq')
ax[2,1].scatter(X_miss_explosion[feature].values, np.ones(len(X_miss_explosion))*2, s = 300, marker = '*',ec = 'k', c = X_miss_explosion['eq_probability'], cmap = 'Reds',label = 'Miss. Exp')

ax[2,1].legend()
#ax[0,0].set_xscale('log')
#ax[0,0].set_xlim(l1,0.02)


ax[2,1].set_title(feature, fontsize = 20)





feature = '0_Median frequency'

l1 = 0
l2 = 0.1

all_df = features_all
data_range = all_df[feature].values.ptp()
num_bins = int((l2 - l1)/ 0.0005)
#sns.histplot(x= no_df[feature].values, bins= num_bins, kde=False, ax = ax[0,0], 
#             multiple='stack', stat = 'percent', color = 'green', alpha = 0.5,label = 'Noise')
#sns.histplot(x= su_df[feature].values, bins= num_bins, kde=False, ax = ax[0,0],   
#             multiple='stack', stat = 'percent', color = 'red', alpha = 0.5, label = 'Surface')
sns.histplot(x= new_eq[feature].values, bins= num_bins, kde=False, ax = ax[2,2],  
             multiple='stack', stat = 'percent',  color = '#1f77b4', alpha = 0.3, label = 'Earthquake')
sns.histplot(x= new_exp[feature].values, bins=num_bins, kde=False, ax = ax[2,2],  
             multiple='stack', stat = 'percent',  color = '#9467bd', alpha = 0.3, label = 'Explosion')



ax[2,2].scatter(X_miss_earthquake[feature].values, np.ones(len(X_miss_earthquake))*1, ec = 'k',s = 300, marker = '^',c = X_miss_earthquake['px_probability'], cmap = 'Reds',label = 'Miss. Eq')
ax[2,2].scatter(X_miss_explosion[feature].values, np.ones(len(X_miss_explosion))*0.5, ec = 'k', s = 300, marker = '*',c = X_miss_explosion['eq_probability'], cmap = 'Reds',label = 'Miss. Exp')


ax[2,2].legend()
#ax[2,2].colorbar()
#ax[0,0].set_xscale('log')
#ax[0,0].set_xlim(l1,0.02)


ax[2,2].set_title(feature, fontsize = 20)

plt.tight_layout()

In [ ]:
st